# Price ML Predicion

## Price Prediction's Caveats

  When the characteristics of the instrument are far from stable, price prediction can be quite off.
  - Examples...
    - SERV
    - MEDS
    - XCUR
  
  Price Prediction should be combined with Prediction of market highs and lows because if you just entered the market based on predicted price, you could get washed out by manipulative Market Wales hitting your stops.
  
  Predicting High, Low, Market Open and Market Close Should be done using separate models, using the same model parameters.
  
  Before trading an instrument, one should review the median market price range, verses median prediction accuracy delta, to verify whether the instrument is worth trading. I have a hunch that the smaller the trading range as compared to the prediction delta, the less likely the instrument is worth engaging with.



In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import yfinance as yf
import pandas_ta as ta
# If the predicted value varies/swings-about too much from the price,
# the days/period training period many be changed to exclude the prior period
# That does not have present-time characteristics with regard to price movement.
data = yf.download(tickers = 'GOOG', start = '2015-06-03',end = '2024-07-21')
offset = pd.Timedelta(days=-30)
# Resample to 'W'eekly or 'ME'(Month End)
# logic = {'Open'  : 'first',
#          'High'  : 'max',
#          'Low'   : 'min',
#          'Close' : 'last',
#          'Adj Close': 'last',
#          'Volume': 'sum'}
# data = data.resample('W', offset=offset).apply(logic)
print(len(data))

data.tail(10)


[*********************100%%**********************]  1 of 1 completed

2298


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2024-07-08,191.365005,191.679001,189.320007,190.479996,190.479996,12097600
2024-07-09,191.750000,192.860001,190.229996,190.440002,190.440002,10198500
2024-07-10,190.750000,193.309998,190.619995,192.660004,192.660004,12052900
2024-07-11,191.339996,192.410004,186.820007,187.300003,187.300003,16452000
2024-07-12,186.919998,188.690002,186.139999,186.779999,186.779999,14449100
2024-07-15,186.490005,189.899994,186.490005,188.190002,188.190002,12186000
2024-07-16,188.960007,190.339996,185.119995,185.500000,185.500000,12760100
2024-07-17,184.679993,185.229996,181.619995,182.619995,182.619995,17376600
2024-07-18,183.539993,184.050003,178.210007,179.220001,179.220001,17877200


# Add indicators to the data


In [32]:
# Add one row at the end.
# A copy of the last row.
pass

# Adding indicators
IndCount = 0
data['RSI']=ta.rsi(data.Close, length=3); IndCount += 1
# data['EMAF']=ta.ema(data.Close, length=3); IndCount += 1
# data['EMAM']=ta.ema(data.Close, length=6); IndCount += 1
# data['EMAS']=ta.ema(data.Close, length=9); IndCount += 1
data['DPO']=ta.dpo(data.Close, length=3, centered=True); IndCount += 1
data['DPO']=ta.dpo(data.Close, length=6, centered=True); IndCount += 1
data['DPO']=ta.dpo(data.Close, length=9, centered=True); IndCount += 1
if data['Volume'].iloc[-1] > 0:
    data = data.join(ta.aobv(data.Close, data.Volume, fast=True, min_lookback=3, max_lookback=9))
    IndCount += 7 # ta.aobv adds 7 columns

data['Target'] = data['Adj Close']-data.Open
data['Target'] = data['Target'].shift(-1)

# data.head(10)

data['TargetClass'] = [1 if data['Target'][i]>0 else 0 for i in range(len(data))]

data['TargetNextClose'] = data['Adj Close'].shift(-1)
# 
data.dropna(inplace=True)
data.reset_index(inplace = True)
# data.drop(['Volume', 'Close', 'Date'], axis=1, inplace=True)
data.drop(['Date'], axis=1, inplace=True)
# Add one more row to the data file, this will be our next day's prediction.
data = pd.concat([data,data[-1:]])
# And, reindex the dataframe.
data.reset_index(inplace=True)

data.tail(10)

/tmp/ipykernel_3700071/1104664637.py:23: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  data['TargetClass'] = [1 if data['Target'][i]>0 else 0 for i in range(len(data))]


,index,Open,High,Low,Close,Adj Close,Volume,RSI,DPO,OBV,OBV_min_3,OBV_max_9,OBVe_1,OBVe_12,AOBV_LR_2,AOBV_SR_2,Target,TargetClass,TargetNextClose
2273,2273,184.479996,185.339996,182.729996,184.490005,184.490005,11815900,55.585015,-2.956662,2.726509e+09,2.714693e+09,2.738075e+09,2.726509e+09,2.687205e+09,0,1,3.139999,1,186.610001
2274,2274,183.470001,186.949997,183.059998,186.610001,186.610001,12555500,71.678412,-1.646667,2.739064e+09,2.714693e+09,2.739064e+09,2.739064e+09,2.695184e+09,1,0,1.089996,1,187.389999
2275,2275,186.300003,187.619995,185.384995,187.389999,187.389999,7409100,76.398120,-0.915558,2.746474e+09,2.726509e+09,2.746474e+09,2.746474e+09,2.703074e+09,1,0,4.639999,1,191.960007
2276,2276,187.320007,192.259995,187.320007,191.960007,191.960007,14303400,90.423551,3.281116,2.760777e+09,2.739064e+09,2.760777e+09,2.760777e+09,2.711952e+09,1,0,-0.885010,0,190.479996
2277,2277,191.365005,191.679001,189.320007,190.479996,190.479996,12097600,70.167856,1.389994,2.748679e+09,2.746474e+09,2.760777e+09,2.748679e+09,2.717602e+09,1,0,-1.309998,0,190.440002
2278,2278,191.750000,192.860001,190.229996,190.440002,190.440002,10198500,69.536475,1.473334,2.738481e+09,2.738481e+09,2.760777e+09,2.738481e+09,2.720814e+09,0,1,1.910004,1,192.660004
2279,2279,190.750000,193.309998,190.619995,192.660004,192.660004,12052900,82.584535,4.223336,2.750534e+09,2.738481e+09,2.760777e+09,2.750534e+09,2.725386e+09,1,0,-4.039993,0,187.300003
2280,2280,191.339996,192.410004,186.820007,187.300003,187.300003,16452000,32.370820,0.278892,2.734082e+09,2.734082e+09,2.760777e+09,2.734082e+09,2.726724e+09,0,1,-0.139999,0,186.779999
2281,2281,186.919998,188.690002,186.139999,186.779999,186.779999,14449100,29.739405,0.991109,2.719633e+09,2.719633e+09,2.760777e+09,2.719633e+09,2.725633e+09,0,1,1.699997,1,188.190002
2282,2281,186.919998,188.690002,186.139999,186.779999,186.779999,14449100,29.739405,0.991109,2.719633e+09,2.719633e+09,2.760777e+09,2.719633e+09,2.725633e+09,0,1,1.699997,1,188.190002


In [33]:
data_set = data.iloc[:, 0:10 + IndCount]#.values
# pd.set_option('display.max_columns', None)

data_set.tail(10)
# print(data_set.shape)
# print(data.shape)
# print(type(data_set))

,index,Open,High,Low,Close,Adj Close,Volume,RSI,DPO,OBV,OBV_min_3,OBV_max_9,OBVe_1,OBVe_12,AOBV_LR_2,AOBV_SR_2,Target,TargetClass,TargetNextClose
2273,2273,184.479996,185.339996,182.729996,184.490005,184.490005,11815900,55.585015,-2.956662,2.726509e+09,2.714693e+09,2.738075e+09,2.726509e+09,2.687205e+09,0,1,3.139999,1,186.610001
2274,2274,183.470001,186.949997,183.059998,186.610001,186.610001,12555500,71.678412,-1.646667,2.739064e+09,2.714693e+09,2.739064e+09,2.739064e+09,2.695184e+09,1,0,1.089996,1,187.389999
2275,2275,186.300003,187.619995,185.384995,187.389999,187.389999,7409100,76.398120,-0.915558,2.746474e+09,2.726509e+09,2.746474e+09,2.746474e+09,2.703074e+09,1,0,4.639999,1,191.960007
2276,2276,187.320007,192.259995,187.320007,191.960007,191.960007,14303400,90.423551,3.281116,2.760777e+09,2.739064e+09,2.760777e+09,2.760777e+09,2.711952e+09,1,0,-0.885010,0,190.479996
2277,2277,191.365005,191.679001,189.320007,190.479996,190.479996,12097600,70.167856,1.389994,2.748679e+09,2.746474e+09,2.760777e+09,2.748679e+09,2.717602e+09,1,0,-1.309998,0,190.440002
2278,2278,191.750000,192.860001,190.229996,190.440002,190.440002,10198500,69.536475,1.473334,2.738481e+09,2.738481e+09,2.760777e+09,2.738481e+09,2.720814e+09,0,1,1.910004,1,192.660004
2279,2279,190.750000,193.309998,190.619995,192.660004,192.660004,12052900,82.584535,4.223336,2.750534e+09,2.738481e+09,2.760777e+09,2.750534e+09,2.725386e+09,1,0,-4.039993,0,187.300003
2280,2280,191.339996,192.410004,186.820007,187.300003,187.300003,16452000,32.370820,0.278892,2.734082e+09,2.734082e+09,2.760777e+09,2.734082e+09,2.726724e+09,0,1,-0.139999,0,186.779999
2281,2281,186.919998,188.690002,186.139999,186.779999,186.779999,14449100,29.739405,0.991109,2.719633e+09,2.719633e+09,2.760777e+09,2.719633e+09,2.725633e+09,0,1,1.699997,1,188.190002
2282,2281,186.919998,188.690002,186.139999,186.779999,186.779999,14449100,29.739405,0.991109,2.719633e+09,2.719633e+09,2.760777e+09,2.719633e+09,2.725633e+09,0,1,1.699997,1,188.190002


# Scale the data

  The data is scaled to a range of 0 to 1.
  This is done to ensure that the data is normalized, so that the model can be trained on it

In [34]:
from sklearn.preprocessing import MinMaxScaler

sc = MinMaxScaler(feature_range=(0,1))
data_set_scaled = sc.fit_transform(data_set)
print("data_set.shape:", data_set.shape, "data_set_scaled.shape", data_set_scaled.shape)
print(data_set_scaled)

data_set.shape: (2283, 19) data_set_scaled.shape (2283, 19)
[[0.00000000e+00 3.46854988e-03 4.61078202e-03 ... 4.66503839e-01
  0.00000000e+00 5.94476175e-03]
 [4.38404209e-04 5.34157718e-03 4.64069750e-03 ... 4.63400881e-01
  0.00000000e+00 6.39377459e-03]
 [8.76808417e-04 6.05941449e-03 6.28270550e-03 ... 4.71275215e-01
  1.00000000e+00 7.07923994e-03]
 ...
 [9.99561596e-01 9.97526746e-01 9.94616402e-01 ... 4.60626585e-01
  0.00000000e+00 9.64758288e-01]
 [1.00000000e+00 9.70864112e-01 9.72364037e-01 ... 5.90143140e-01
  1.00000000e+00 9.73209121e-01]
 [1.00000000e+00 9.70864112e-01 9.72364037e-01 ... 5.90143140e-01
  1.00000000e+00 9.73209121e-01]]


# Prepare the scaled data for the model

  - The data is prepared for the model by creating a 3D array of the data.
  - The data is split into training and test data.
  - The training data is used to train the model.
  - The test data is used to test the model.
  - The model is then used to predict the next day's price.

In [35]:
# multiple feature from data provided to the model
X = []
# print(data_set_scaled[0].size)
# data_set_scaled=data_set.values
backcandles = 15
print(data_set_scaled.shape[0])

for j in range(8): # data_set_scaled[0].size):#2 columns are target not X
    X.append([])
    for i in range(backcandles, data_set_scaled.shape[0]): # backcandles+2
        X[j].append(data_set_scaled[i-backcandles:i, j])

#move axis from 0 to position 2
# Converts the list of arrays into a 3D ndarray array.
X=np.moveaxis(X, [0], [2])
# X is not used.

#Erase first elements of y because of backcandles to match X length
#del(yi[0:backcandles])
#X, yi = np.array(X), np.array(yi)
# Choose -1 for last column, classification else -2...
X, yi =np.array(X), np.array(data_set_scaled[backcandles:,-1])
y=np.reshape(yi,(len(yi),1))
#y=sc.fit_transform(yi)
#X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))

print("data_set.shape:",data_set.shape,"X.shape:",X.shape)
print(X)
print("=========================================================")
print(y.shape)
print(y)


2283
data_set.shape: (2283, 19) X.shape: (2268, 15, 8)
[[[0.00000000e+00 3.46854988e-03 4.61078202e-03 ... 5.95674213e-03
   1.37390115e-01 7.79639083e-01]
  [4.38404209e-04 5.34157718e-03 4.64069750e-03 ... 5.94476175e-03
   1.42973350e-01 7.75717636e-01]
  [8.76808417e-04 6.05941449e-03 6.28270550e-03 ... 6.39377459e-03
   8.35174384e-02 8.24830857e-01]
  ...
  [5.26085050e-03 1.09485436e-03 1.03066902e-03 ... 2.45154290e-03
   1.15584067e-01 5.71584842e-01]
  [5.69925471e-03 4.67497450e-04 0.00000000e+00 ... 0.00000000e+00
   8.78065464e-02 1.72358418e-01]
  [6.13765892e-03 1.09183988e-03 3.09856746e-04 ... 1.15243733e-03
   1.38240541e-01 4.40364458e-01]]

 [[4.38404209e-04 5.34157718e-03 4.64069750e-03 ... 5.94476175e-03
   1.42973350e-01 7.75717636e-01]
  [8.76808417e-04 6.05941449e-03 6.28270550e-03 ... 6.39377459e-03
   8.35174384e-02 8.24830857e-01]
  [1.31521263e-03 6.07449840e-03 5.61243691e-03 ... 7.07923994e-03
   7.86367292e-02 8.83771119e-01]
  ...
  [5.69925471e-03 4.67

In [36]:
#also comprehensions for X
#X = np.array([data_set_scaled[i-backcandles:i,:4].copy() for i in range(backcandles,len(data_set_scaled))])
#print(X)
#print(X.shape)

In [37]:
# split data into train test sets
splitlimit = int(len(X)*0.8)
print("lenX:",len(X), "splitLimit:",splitlimit)
X_train, X_test = X[:splitlimit], X[splitlimit:] # Training data, Test Data
y_train, y_test = y[:splitlimit], y[splitlimit:] # Training data, Test Data
adj_close_train, adj_close_test = data_set.loc[:splitlimit, 'Adj Close':'Adj Close'], data_set.loc[splitlimit:, 'Adj Close':'Adj Close'] # Training data, Test Data
print("X_train.shape:", X_train.shape)
print("y_train.shape:", y_train.shape)
print("X_test.shape:", X_test.shape)
print("y_test.shape:", y_test.shape)
print("adj_close_test.shape:", adj_close_test.shape)
print("=============================================")
print("y_train:", y_train)

lenX: 2268 splitLimit: 1814
X_train.shape: (1814, 15, 8)
y_train.shape: (1814, 1)
X_test.shape: (454, 15, 8)
y_test.shape: (454, 1)
adj_close_test.shape: (469, 1)
y_train: [[0.0088962 ]
 [0.0132515 ]
 [0.01298808]
 ...
 [0.45491816]
 [0.44402242]
 [0.44737496]]


# Model Training

In [38]:
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers import Dense
from keras.layers import TimeDistributed

import tensorflow as tf
import keras
from keras import optimizers
from keras.callbacks import History
from keras.models import Model
from keras.layers import Dense, Dropout, LSTM, Input, Activation, concatenate
import numpy as np
#tf.random.set_seed(20)
np.random.seed(10)

lstm_input = Input(shape=(backcandles, 8), name='lstm_input')
inputs = LSTM(200, name='first_layer')(lstm_input)
inputs = Dense(1, name='dense_layer')(inputs)
output = Activation('linear', name='output')(inputs)
model = Model(inputs=lstm_input, outputs=output)
adam = optimizers.Adam()
model.compile(optimizer=adam, loss='mse')
# model.fit(x=X_train, y=y_train, batch_size=15, epochs=30, shuffle=True, validation_split = 0.1)
fitted_model = model.fit(x=X_train, y=y_train, batch_size=15, epochs=50, shuffle=True, validation_split = 0.1)
print("adj_close_test.shape:", adj_close_test.shape)
pass

Epoch 1/50
109/109 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.0109 - val_loss: 0.0016
Epoch 2/50
109/109 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 1.2799e-04 - val_loss: 7.7895e-04
Epoch 3/50
109/109 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 1.2336e-04 - val_loss: 6.8748e-04
Epoch 4/50
109/109 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 1.4067e-04 - val_loss: 7.8862e-04
Epoch 5/50
109/109 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 1.1818e-04 - val_loss: 8.7181e-04
Epoch 6/50
109/109 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 1.1181e-04 - val_loss: 8.1110e-04
Epoch 7/50
109/109 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 1.2129e-04 - val_loss: 8.4458e-04
Epoch 8/50
109/109 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 1.3799e-04 - val_loss: 9.7260e-04
Epoch 9/50
109/109 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 1.2300e-04 - val_loss: 7.9859e-04
Epoch 10/50
109/109 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 1.3594e-04 - val_loss: 6.4985e-04
Epoch 11/50
109/109 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 1.3628

# Model Testing

In [39]:
y_pred = model.predict(X_test)
# y_pred_rescaled = sc.inverse_transform(y_pred)
#y_pred=np.where(y_pred > 0.43, 1,0)
for i in range(10):
    print(y_pred[i], y_test[i])

15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
[0.4594844] [0.43899358]
[0.4586434] [0.4368384]
[0.44923368] [0.43252797]
[0.44622824] [0.44839265]
[0.44395146] [0.43252797]
[0.45898914] [0.42091386]
[0.4415273] [0.4397719]
[0.43379387] [0.45839046]
[0.44719696] [0.45725297]
[0.46222612] [0.45737268]


# Adjust the prediction
  The ideal for the prediction is for it to be as close to the actual price as possible.
  What was observed is that the prediction is often off by a delta.
  The delta is the difference between the actual price and the predicted price.
  After applying an average of the delta to the prediction, the prediction is closer to the actual price.
  Usually within 1% of the actual price, which is an incredible improvement.  
  
  The resulting numpy array y_p_adj is the adjusted prediction, which we will use as the prediction for the next day.

In [40]:
# Calculate the delta between actual price and prediction
# Bring the prediction closer to the price based on the delta
def mov_avg(x, w):
    return abs(np.convolve(x, np.ones(w), 'valid') / w)

def apply_deltas(a,p,d, w):
    al = list(a)
    pl = list(p)
    dl = list(d)
    rl = []
    i = w - 1
    for j in range(0, i):
        rl.append(pl[j]) 
    for delta in dl:
        if al[i] > pl[i]:
            val = pl[i] + delta
        else:
            val = pl[i] - delta
        rl.append(val)
        i += 1
    return rl

avg_win = 6

y_p_delta = y_pred - y_test 
y_pd_avg = mov_avg(y_p_delta[:, 0], avg_win)
# print(y_test.shape, y_pred.shape, y_p_adj.shape, y_p_delta.shape)
y_p_adj = apply_deltas(y_test, y_pred, y_pd_avg, avg_win)
# We don't know the actual price for today, so we don't know the delta and average,
# so for today, we will use the last known average delta to adjust todays prediction.
if y_test[-2] > y_pred[-1]:
    y_p_adj[-1] = y_pred[-1] + y_pd_avg[-2]
else:
    y_p_adj[-1] = y_pred[-1] - y_pd_avg[-2]
# Convert y_p_adj to a numpy array for plotting.
y_p_adj = np.array(y_p_adj)

print("adj_close_test.shape:", adj_close_test.shape)
print("y_pred:", y_pred.shape[0])


adj_close_test.shape: (469, 1)
y_pred: 454


# Plot the Test Results

In [41]:
# Use an interactive plot to see the results
%matplotlib notebook
# Plot the scaled test and predicted data.
y_test_plot = y_test[:-1]
plt.figure(figsize=(16,8))
plt.plot(y_test_plot, color = 'black', label = 'Test', marker='.')
# plt.plot(y_pred, color = 'red', label = 'Pred', marker='1')
plt.plot(y_p_adj, color = 'green', label = 'Adj Pred', marker='1')
plt.legend()
plt.grid()
plt.show()


<IPython.core.display.Javascript object>

In [42]:
# Use an interactive plot to see the results
%matplotlib notebook

# Plot the original Adj Cost data that is in dollars...
# Remove the last Adj Price, because it just aopy of the last adj
plt_test = np.array(data_set[15 + splitlimit:]['Adj Close'])
plt_test_plot = plt_test[:-1]

# Rescale the predicted data to dollars...
plt_pred_adj = sc.inverse_transform(np.append(y_p_adj, data_set.to_numpy()[15 + splitlimit:, 1:], axis=1))[:, -1]
plt_test_scaler = np.array( pd.DataFrame([ plt_test[i] / y_test[i] for i in range(len(plt_test)) ]).rolling(window=9).mean() )
plt_pred_adj = [ ((y_p_adj[i] * plt_test_scaler[i]) - (y_test[i] * plt_test_scaler[i])) + plt_test[i] for i in range(len(y_p_adj)) ] 

pass

# empty_arr = np.zeros(((len(plt_test_plot)+1, 18)))
# plt_pred_adj = sc.inverse_transform(np.append(y_p_adj, empty_arr, axis=1))[0:, 1]

# Plot the data...
plt.figure(figsize=(16,8))
plt.plot(plt_test_plot, color = 'black', label = 'Test', marker='.')
# plt.plot(y_pred, color = 'red', label = 'Pred', marker='1')
plt.plot(plt_pred_adj, color = 'green', label = 'Adj Pred', marker='1')
plt.grid()
plt.legend()
plt.show()

pass

<IPython.core.display.Javascript object>

In [43]:
# We convert our data back to dollars...
# predval = y_pred[i] * scalefac

# Convert back to dollar $values
elements = len(y_pred)
print(elements)
tot_deltas = 0
tot_tradrng = 0
for i in range(-1, -elements, -1):
    actual = data_set['Adj Close'].iloc[i - 1]
    scalefac = data_set['Adj Close'].iloc[i - 1] / y_test[i - 1]
    # print("Scaling Factor", scalefac)
    predval = y_pred[i] * scalefac
    predval = y_pred[i - 1] if np.isinf(predval) else predval
    pred_delta = abs(predval - actual)
    tot_deltas += pred_delta
    trd_rng = abs(data_set['High'].iloc[i - 1] - data_set['Low'].iloc[i])
    tot_tradrng += trd_rng
    print("Predicted: ", predval, "  Actual:", actual, "  Delta:", pred_delta, "  Trade Rng:", trd_rng)
print("Mean Trading Range:", round(tot_tradrng / elements, 2))
print("Mean Delta:", (tot_deltas / elements).rnd(2))

454
Predicted:  [188.87047821]   Actual: 186.77999877929688   Delta: [2.09047943]   Trade Rng: 2.5500030517578125
Predicted:  [190.58126571]   Actual: 187.3000030517578   Delta: [3.28126266]   Trade Rng: 6.2700042724609375
Predicted:  [201.12034409]   Actual: 192.66000366210938   Delta: [8.46034043]   Trade Rng: 6.489990234375
Predicted:  [190.43892409]   Actual: 190.44000244140625   Delta: [0.00107836]   Trade Rng: 2.2400054931640625
Predicted:  [191.2556199]   Actual: 190.47999572753906   Delta: [0.77562417]   Trade Rng: 1.449005126953125
Predicted:  [193.3811506]   Actual: 191.9600067138672   Delta: [1.42114388]   Trade Rng: 2.9399871826171875
Predicted:  [184.13679699]   Actual: 187.38999938964844   Delta: [3.2532024]   Trade Rng: 0.29998779296875
Predicted:  [187.96887196]   Actual: 186.61000061035156   Delta: [1.35887135]   Trade Rng: 1.56500244140625
Predicted:  [185.04844141]   Actual: 184.49000549316406   Delta: [0.55843592]   Trade Rng: 2.279998779296875
Predicted:  [183.4128

AttributeError: 'numpy.ndarray' object has no attribute 'rnd'

In [ ]:
elements = len(y_p_adj)
print(elements)
tot_deltas = 0
tot_tradrng = 0
for i in range(-1, -elements, -1):
    actual = data_set['Adj Close'].iloc[i - 1]
    scalefac = data_set['Adj Close'].iloc[i - 1] / y_test[i - 1]
    # print("Scaling Factor", scalefac)
    predval = y_p_adj[i] * scalefac
    predval = y_pred[i - 1] if np.isinf(predval) else predval
    pred_delta = abs(predval - actual)
    tot_deltas += pred_delta
    trd_rng = abs(data_set['High'].iloc[i - 1] - data_set['Low'].iloc[i])
    tot_tradrng += trd_rng
    print("Predicted: ", predval, "  Actual:", actual, "  Delta:", pred_delta, "  Trade Rng:", trd_rng)
print("Mean Trading Range:", round(tot_tradrng / elements, 2))
print("Mean Delta:", round((tot_deltas[0] / elements), 2))

In [ ]:
# Calculate an array of deltas between predicted price and actual price
print("Std Dev -     Pred: ", y_p_delta.std())
y_pa_delta = abs(y_test - y_p_adj)
print("Std Dev - Pred Adj: ", y_pa_delta.std())
print("Better by", (y_p_delta.std() / y_pa_delta.std()), "Standard Deviations")

In [ ]:
# How often is the prediction within 1% of the actual price?
pct = 0.05
pct_delta = abs(y_test * pct)
delta = abs(y_test - y_p_adj)
delta = delta[delta < pct_delta]
# Get average price range for the instrument
avg_price_rng = data_set['High'] - data_set['Low']
# Get pct of average price range
pcnt_avg_price_rng = avg_price_rng.mean() * pct

print("Within",pct * 100,"%:","[ Trading Range:$", "Within Avg Price:", avg_price_rng.mean().round(2),pcnt_avg_price_rng.round(2),"]", round((len(delta) / len(y_test)),2))

# How often is the predicted trend the same as the actual trend?
test_trend = y_test[1:] - y_test[:-1]
pred_trend = y_p_adj[1:] - y_p_adj[:-1]
delta = [1 if (test_trend < 0) and (pred_trend < 0) or (test_trend > 0) and (pred_trend > 0) else 0 for test_trend, pred_trend in zip(test_trend, pred_trend)]
# Print the number of elements in delta and the percent of delta that are 1.
print("Same Trend:", len(delta), "Wrong:", len(delta) - np.count_nonzero(delta), "Right", np.count_nonzero(delta), "%Right", round(sum(delta) / len(delta) * 100,2))
